In [1]:
import pandas as pd
import re
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import csv

import duckdb

conn = duckdb.connect('/global/scratch/users/empchase/Seq_analysis/mydb.db') # create an in-memory database

In [2]:
conn.sql("SHOW ALL TABLES")

┌──────────┬─────────┬─────────────┬───────────────────────────┬──────────────────────────────────────┬───────────┐
│ database │ schema  │    name     │       column_names        │             column_types             │ temporary │
│ varchar  │ varchar │   varchar   │         varchar[]         │              varchar[]               │  boolean  │
├──────────┼─────────┼─────────────┼───────────────────────────┼──────────────────────────────────────┼───────────┤
│ mydb     │ main    │ SI_VI_AD0   │ [index, BCs, TBB, Tiles]  │ [VARCHAR, BIGINT, VARCHAR, VARCHAR]  │ false     │
│ mydb     │ main    │ SI_VI_AD1   │ [index, BCs, TBB, Tiles]  │ [VARCHAR, BIGINT, VARCHAR, VARCHAR]  │ false     │
│ mydb     │ main    │ SI_VI_RPTR0 │ [index, BCs, TBB, Tiles]  │ [VARCHAR, BIGINT, VARCHAR, VARCHAR]  │ false     │
│ mydb     │ main    │ SI_VI_RPTR1 │ [index, BCs, TBB, Tiles]  │ [VARCHAR, BIGINT, VARCHAR, VARCHAR]  │ false     │
│ mydb     │ main    │ TBB         │ [Tiles, ADbc, RPTRbc]     │ [VARCHA

In [3]:
pwd

'/global/scratch/users/empchase/CiberVII/CiberVII_CZB/EC_Demultiplexed'

In [9]:
#where are the reads
path = '/global/scratch/users/empchase/CiberVII/CiberVII_CZB/EC_Demultiplexed/'

directory = os.fsencode(path)
AD1files = []
RPTR1files = []

for file in os.listdir(directory):
    filename = os.fsdecode(file)
#     x = filename.split('_')
#     filename = path + filename
    
    if filename.endswith('.fastq'):
        x = filename.split('_')
#         print(x)
        if 'AD' in x:
            AD1files.append(filename)
        elif 'RPTR' in x:
            RPTR1files.append(filename)
            
print(AD1files)
print(RPTR1files)

['AD_4_30_16X16_0925_S12_L004_R1_001.fastq', 'AD_4_30_30X20_S206_L003_R1_001_demultiplexed.fastq', 'AD_4_15_20X20_S8_L004_R1_001.fastq', 'AD_maxiprep_20X20_S9_L004_R1_001.fastq', 'AD_4_30_16X16_0927_S205_L003_R1_001_demultiplexed.fastq', 'AD_3_30_30X20_S206_L004_R1_001_demultiplexed.fastq', 'AD_3_30_16X16_S13_L004_R1_001.fastq']
['RPTR_maxiprep_20X20_S11_L004_R1_001.fastq', 'RPTR_4_30_30X20_S207_L003_R1_001_demultiplexed.fastq', 'RPTR_4_30_16X16_0925_S14_L004_R1_001.fastq', 'RPTR_3_30_16X16_S15_L004_R1_001.fastq', 'RPTR_4_15_20X20_S10_L004_R1_001.fastq', 'RPTR_3_30_30X20_S207_L004_R1_001_demultiplexed.fastq']


In [10]:
def complement(seq):
    complement = {'A': 'T', 'C': 'G', 'G': 'C', 'T': 'A', 'N':'N', 'X':'X'} 
    bases = list(seq) 
    bases = [complement[base] for base in bases] 
    return ''.join(bases)
def reverse_complement(s):
        return complement(s[::-1])

In [11]:
def getmid(seq, pre, post, bclen):
    # seq = the sequence to parse
    # pre = substring that precedes piece of interest
    # post = substring that follows piece of interest
    # returns piece of interest

    re_key = pre + "(.*)"+ post 
    poi_search = re.search(re_key, seq)
#     print(poi_search)
#     poi = ''


    if poi_search is None:
        #the barcode will be called X
        poi = "X"
        
        #then we search for which restriction site had the error
        #regex for the bc we want to ignore
        w = "(.{"+str(bclen)+"})" 
#         print(w)
        pre_re = pre + w + "(.{7})"
        pre_search = re.search(pre_re, seq)
#         print(pre_search)
        post_re = "(\w{7})" + w + post
        post_search = re.search(post_re, seq)
#         print(post_search)
        
        if pre_search is None and post_search is None:
            a = 'X'
            z = 'X'
        elif pre_search is None:
            poi = post_search.group(2)
            a = post_search.group(1)
            z = post
        elif post_search is None:
            poi = pre_search.group(1)
            z = pre_search.group(2)
            a = pre
        else:
            a = "Z"
            z = 'Z'
#             print(seq)
            
    else:
        poi = poi_search.group(1)
        a = pre
        z = post
    
    return poi, a, z






#putative consensus sequences ***reverse complement of snapgene***
adp = 'CGGGCCC'#7 bp ; beforeAD barcode in read1
adf = 'GGCGCGC' #7bp ; after AD barcode in read1

rpp = 'AGCGGCC' #7bp ; before rptr barcode in read1
rpf = 'CTCGAGT' #7 bp ; after rptr barcode in read1


# function that just looks for bc
def bc_finder(readfile,bc_pre, bc_post, bc_len):

    # make lists of reads
    seqlist = []
    
    with open(readfile, 'r') as fin:
        for line in fin:
            if line.startswith('@'):
                #look at next line to get read sequence, add to list
                seq = next(fin)
                seq = seq.strip()
                seqlist.append(seq)

    
    #make lists of BCs from list of reads
    bc_list = []
    bc_lens = []
    q_list = []
    
    prex_list = []
    prex_len = []
    postx_list = []
    postx_len = []
       
    for read in seqlist:
   
        bc, prex, postx = getmid(read, bc_pre, bc_post, bc_len)
        bc = reverse_complement(bc) #return reverse complement
        bc_list.append(bc)
        bcl = len(bc)
        bc_lens.append(bcl)
        
        prex_list.append(prex)
        prex_len.append(len(prex))
        postx_list.append(postx)
        postx_len.append(len(postx))
        if bcl == bc_len:
            q_list.append(1)
        else:
            q_list.append(0)
    #         print(bc)

    
    BC_dict = {"BCs":bc_list, "Length":bc_lens,"Qual":q_list, 
               "prex":prex_list, 'prexl':prex_len,
               'postx':postx_list, 'postxl':postx_len} #not lengths but matches to rpp and rpf
    BC_df = pd.DataFrame.from_dict(BC_dict)
    
    #label BC with library name
    
    libname = '_'.join(readfile.split('/')[-1].split('_')[0:6])
    BC_df['Library'] = libname
    
    return BC_df

In [12]:
# function that just looks for bc
def bc_finder_seqonly(readfile,bc_pre, bc_post, bc_len):

    # make lists of reads
    seqlist = []
    
    with open(readfile, 'r') as fin:
        for line in fin:
            seq = line.strip()
            seqlist.append(seq)

    
    #make lists of BCs from list of reads
    bc_list = []
    bc_lens = []
    q_list = []
    
    prex_list = []
    prex_len = []
    postx_list = []
    postx_len = []
       
    for read in seqlist:
   
        bc, prex, postx = getmid(read, bc_pre, bc_post, bc_len)
        bc = reverse_complement(bc) #return reverse complement
        bc_list.append(bc)
        bcl = len(bc)
        bc_lens.append(bcl)
        
        prex_list.append(prex)
        prex_len.append(len(prex))
        postx_list.append(postx)
        postx_len.append(len(postx))
        if bcl == bc_len:
            q_list.append(1)
        else:
            q_list.append(0)
    #         print(bc)

    
    BC_dict = {"BCs":bc_list, "Length":bc_lens,"Qual":q_list, 
               "prex":prex_list, 'prexl':prex_len,
               'postx':postx_list, 'postxl':postx_len} #not lengths but matches to rpp and rpf
    BC_df = pd.DataFrame.from_dict(BC_dict)
    
    #label BC with library name
    
    libname = '_'.join(readfile.split('/')[-1].split('_')[0:6])
    BC_df['Library'] = libname
    
    return BC_df

In [13]:
#read in as df and create dictionary
tbbdf = conn.sql("""SELECT * FROM TBB_UNION""").df()
tbbdf.head()

,Tile,ADbc,RPTRbc,TBB
0,GATTTGCAAGGTAAATTCTTGGCTGCTCCATTGGAAGAGAATCCAA...,TTTATCCTCGG,AAAAAATTACTTGG,GATTTGCAAGGTAAATTCTTGGCTGCTCCATTGGAAGAGAATCCAA...
1,GACCCATATATGTCTGCTCCAAATTCTACTGCATTTACTGCTTTGA...,TGAAATGGTAA,CAAAAAGCTATGGT,GACCCATATATGTCTGCTCCAAATTCTACTGCATTTACTGCTTTGA...
2,GCTTTGGCTGGTGATCCATGGTATCCATTGTTTCCACAAGATGATC...,TGGTAATGAGA,TTTAAATGGCTAAT,GCTTTGGCTGGTGATCCATGGTATCCATTGTTTCCACAAGATGATC...
3,GATTATCATTCTCAAGCTACTGCTGACTTTGTCTTGTATCCACAAG...,AAGGTCCGAGG,AACAAATACAGAGT,GATTATCATTCTCAAGCTACTGCTGACTTTGTCTTGTATCCACAAG...
4,GATATTGAAGCTAGATCATTCTATGTCTTTGGTTCTCCAGTTTCTC...,CCCCGCCGTCG,TCTAGGTTTCACCC,GATATTGAAGCTAGATCATTCTATGTCTTTGGTTCTCCAGTTTCTC...


In [14]:
# t1 = bc_finder('RPTR_maxiprep_test.fastq', rpp, rpf, 14)
# t1

FileNotFoundError: [Errno 2] No such file or directory: 'RPTR_maxiprep_test.fastq'

In [15]:
def analyze_bcs (df, bc_len, bcdict):
    # df = barcode containing df, parsed from fastq
    # bc_len = int, expected barcode length
    # tbb_dictkey = str, either 'ADbc' or 'RPTRbc'
    
    print(df.loc[0,'Library'])
    print()
    print('total reads')
    print(df.shape[0]) #how many total reads
    
    cls = df[df['Length']== bc_len] #cl = correct length
    print('BCs of correct length')
    print(cls.shape[0])
    
    print('% BCs of correct length')
    print (cls.shape[0]/df.shape[0])
    
    #df of BC coverage
    cl_covdf = cls['BCs'].value_counts().to_frame().reset_index() 
    print('# Unique BCs')
    print(cl_covdf.shape[0])
#     print('SUM Unique BCs')
#     print(cl_covdf.sum(numeric_only=True)['BCs'])

    #copy down unique BCs into a list
    bcs = cl_covdf['index'].tolist()
    

    
    
    print('# BC matches to A10 deep seq map')

    matchlist = [] #list of TBBs that match the BCs
#     matches = 0
    for x in bcs:
        try:
            tbb = bcdict[x]
        except KeyError:
            matchlist.append(0)
        else:
            matchlist.append(tbb)
#             matches += 1
            

            
    cl_covdf['TBB'] = matchlist
    
    matchesonly = cl_covdf.replace(0, np.nan)
    matchesonly = matchesonly.dropna()
    totmatches = matchesonly.sum(numeric_only=True)['BCs']
    
#     print(matches)
    print(matchesonly.shape[0])
    
    print('TOT BC matches to A10 deep seq map')
    print(totmatches)
    
#     print('% unique BCs matched')
#     print(totmatches/cl_covdf.shape[0])
    
    
    print()
    
    #label the df with library name
    
    cl_covdf['Library'] = df.loc[0,'Library']
    
    return cl_covdf
    

In [16]:
#dictionary ~ {RPTRbc: TBB}

RPTRdict = dict(zip(tbbdf.RPTRbc, tbbdf.TBB))

In [17]:
#dictionary ~ {ADbc: TBB}

A10ADdict = dict(zip(tbbdf.ADbc, tbbdf.TBB)) #this function overwrites the value of any repeated key. 

In [36]:
#Test
# print(Rlist[2])
shorttest = analyze_bcs(t1, 14, RPTRdict)
shorttest.head()

RPTR_maxiprep_test.fastq

total reads
5
BCs of correct length
5
% BCs of correct length
1.0
# Unique BCs
3
# BC matches to A10 deep seq map
1
TOT BC matches to A10 deep seq map
2



,index,BCs,TBB,Library
0,CTCAATCGACAATT,2,TTTCCAACTGCTAATATTCCATCTGAGTTCTTTAATTCATTTACTA...,RPTR_maxiprep_test.fastq
1,TGGAGAAAGAAAAT,2,0,RPTR_maxiprep_test.fastq
2,GTGATCGGTAACAC,1,0,RPTR_maxiprep_test.fastq


It works ! When I come back try all:

In [18]:
# make df out of each fastq file
Alist = []
Rlist = []

for file in AD1files:
    x = bc_finder(file, adp, adf, 11)
    Alist.append(x)


for file in RPTR1files:
    x = bc_finder(file, rpp, rpf, 14)
    Rlist.append(x)

In [19]:
# Rlist.pop(6)
len(Rlist)

6

In [20]:
tmp = bc_finder_seqonly('RPTR_4_30_16X16_sequences_R1_001.txt', rpp, rpf, 14)

In [21]:
tmp[tmp['Qual']==1]

,BCs,Length,Qual,prex,prexl,postx,postxl,Library
379,TGGACCGCGGGTTT,14,1,AGCGGCC,7,CTCTATA,7,RPTR_4_30_16X16_sequences_R1
685,ACCGCGTGACAGTT,14,1,AGCGGCC,7,CCACTCT,7,RPTR_4_30_16X16_sequences_R1
3558,GTTAGCTAGCATGG,14,1,TGAGCAT,7,CTCGAGT,7,RPTR_4_30_16X16_sequences_R1
3894,ATAGAGTGGACCGC,14,1,AGCGGCC,7,AACGCAC,7,RPTR_4_30_16X16_sequences_R1
4063,ACCGCGTGTATGCC,14,1,AGCGGCC,7,CCACTCT,7,RPTR_4_30_16X16_sequences_R1
...,...,...,...,...,...,...,...,...
2361841,AACGCGGTATCAGA,14,1,AGCGGCC,7,CCACTCT,7,RPTR_4_30_16X16_sequences_R1
2362410,AGTGGACCGCGCGG,14,1,AGCGGCC,7,CTATAAC,7,RPTR_4_30_16X16_sequences_R1
2362480,TTCAGACGTGTGCT,14,1,TCGGAAG,7,CTCGAGT,7,RPTR_4_30_16X16_sequences_R1
2362493,TTCAGACGTGTGCT,14,1,TCGGAAG,7,CTCGAGT,7,RPTR_4_30_16X16_sequences_R1


In [22]:
Rlist.append(tmp)

In [23]:
def split_TBB (countsdf):
    # finds the putative tiles and ADs via the TBB and adds it to the df IN PLACE
    putative_tiles = []
    putative_ads = []

    for index, row in countsdf.loc[:, ['TBB']].iterrows():
        try:
            #     print(row['TBB'])
            tbb_str = row['TBB']
            ptile = tbb_str.split('-')[0]
            pad = tbb_str.split('-')[1]
        except AttributeError:
            ptile = 0
            pad = 0 
        finally:
            putative_tiles.append(ptile)
            putative_ads.append(pad)

    countsdf['Putative_Tile'] = putative_tiles
    countsdf['Putative_AD'] = putative_ads

In [24]:
Rlist[6].head()

,BCs,Length,Qual,prex,prexl,postx,postxl,Library
0,X,1,0,X,1,X,1,RPTR_4_30_16X16_sequences_R1
1,X,1,0,X,1,X,1,RPTR_4_30_16X16_sequences_R1
2,X,1,0,X,1,X,1,RPTR_4_30_16X16_sequences_R1
3,X,1,0,X,1,X,1,RPTR_4_30_16X16_sequences_R1
4,X,1,0,X,1,X,1,RPTR_4_30_16X16_sequences_R1


In [25]:
#analyze all dfs and get dictionary of ADs:TBBs from each df
#the dfs we keep in the end in analyzed_rptr_dfs are not clean (have 0s and RPTR BCs not matching to A10)

analyzed_rptr_dfs = []
ADdict = {}

for og_df in Rlist:
    # get the counts dfs, split the TBBs in each
    counts_df = analyze_bcs(og_df, 14, RPTRdict)
    split_TBB(counts_df)
    analyzed_rptr_dfs.append(counts_df)
    
    counts_df_clean = counts_df.replace(0, np.nan)
    counts_df_clean = counts_df_clean.dropna()
    ad_dict = dict(zip(counts_df_clean.Putative_AD, counts_df_clean.TBB))
    ADdict.update(ad_dict)

RPTR_maxiprep_20X20_S11_L004_R1

total reads
28234403
BCs of correct length
28031839
% BCs of correct length
0.992825631907287
# Unique BCs
27348
# BC matches to A10 deep seq map
5199
TOT BC matches to A10 deep seq map
17731610

RPTR_4_30_30X20_S207_L003

total reads
8003931
BCs of correct length
4672815
% BCs of correct length
0.5838150029029485
# Unique BCs
5198
# BC matches to A10 deep seq map
456
TOT BC matches to A10 deep seq map
1010823

RPTR_4_30_16X16_0925_S14

total reads
10056100
BCs of correct length
4861784
% BCs of correct length
0.4834661548711727
# Unique BCs
3183
# BC matches to A10 deep seq map
270
TOT BC matches to A10 deep seq map
349967

RPTR_3_30_16X16_S15_L004

total reads
23270817
BCs of correct length
20820083
% BCs of correct length
0.8946863790815768
# Unique BCs
9879
# BC matches to A10 deep seq map
822
TOT BC matches to A10 deep seq map
13042629

RPTR_4_15_20X20_S10_L004

total reads
1908929
BCs of correct length
1223180
% BCs of correct length
0.64076767653

In [26]:
len(ADdict)

6040

In [27]:
#analyze AD BCs
analyzed_ad_dfs = []
counter_ADdict = {}

for og_df in Alist:
    # get the counts dfs, split the TBBs in each
    counts_df = analyze_bcs(og_df, 11, ADdict)
    split_TBB(counts_df)
    analyzed_ad_dfs.append(counts_df)
    
    counts_df_clean = counts_df.replace(0, np.nan)
    counts_df_clean = counts_df_clean.dropna()
    ad_dict = dict(zip(counts_df_clean.Putative_AD, counts_df_clean.TBB))
    counter_ADdict.update(ad_dict)

AD_4_30_16X16_0925_S12

total reads
25195610
BCs of correct length
23626165
% BCs of correct length
0.9377095851221701
# Unique BCs
5495
# BC matches to A10 deep seq map
16
TOT BC matches to A10 deep seq map
77

AD_4_30_30X20_S206_L003

total reads
12475045
BCs of correct length
12431038
% BCs of correct length
0.9964723974943578
# Unique BCs
1372
# BC matches to A10 deep seq map
3
TOT BC matches to A10 deep seq map
9

AD_4_15_20X20_S8_L004

total reads
4067651
BCs of correct length
4059197
% BCs of correct length
0.9979216506037514
# Unique BCs
2224
# BC matches to A10 deep seq map
2
TOT BC matches to A10 deep seq map
26641

AD_maxiprep_20X20_S9_L004_R1

total reads
7353104
BCs of correct length
7294463
% BCs of correct length
0.9920250005983867
# Unique BCs
305788
# BC matches to A10 deep seq map
3312
TOT BC matches to A10 deep seq map
180031

AD_4_30_16X16_0927_S205

total reads
1251
BCs of correct length
5
% BCs of correct length
0.003996802557953637
# Unique BCs
4
# BC matches to 

In [28]:
#analyze AD BCs
A10analyzed_ad_dfs = []
counter_ADdict_A10 = {}

for og_df in Alist:
    # get the counts dfs, split the TBBs in each
    counts_df = analyze_bcs(og_df, 11, A10ADdict)
    split_TBB(counts_df)
    A10analyzed_ad_dfs.append(counts_df)
    
    counts_df_clean = counts_df.replace(0, np.nan)
    counts_df_clean = counts_df_clean.dropna()
    ad_dict = dict(zip(counts_df_clean.Putative_AD, counts_df_clean.TBB))
    counter_ADdict_A10.update(ad_dict)

AD_4_30_16X16_0925_S12

total reads
25195610
BCs of correct length
23626165
% BCs of correct length
0.9377095851221701
# Unique BCs
5495
# BC matches to A10 deep seq map
5147
TOT BC matches to A10 deep seq map
23612898

AD_4_30_30X20_S206_L003

total reads
12475045
BCs of correct length
12431038
% BCs of correct length
0.9964723974943578
# Unique BCs
1372
# BC matches to A10 deep seq map
1262
TOT BC matches to A10 deep seq map
12428562

AD_4_15_20X20_S8_L004

total reads
4067651
BCs of correct length
4059197
% BCs of correct length
0.9979216506037514
# Unique BCs
2224
# BC matches to A10 deep seq map
2094
TOT BC matches to A10 deep seq map
3952494

AD_maxiprep_20X20_S9_L004_R1

total reads
7353104
BCs of correct length
7294463
% BCs of correct length
0.9920250005983867
# Unique BCs
305788
# BC matches to A10 deep seq map
290462
TOT BC matches to A10 deep seq map
6965551

AD_4_30_16X16_0927_S205

total reads
1251
BCs of correct length
5
% BCs of correct length
0.003996802557953637
# Uni

In [29]:
#save useful tables into database

# conn.sql("CREATE TABLE SI_VI_RPTR0 AS SELECT * FROM rptrmatch0")
# conn.sql("CREATE TABLE SI_VI_RPTR1 AS SELECT * FROM rptrmatch1")
# conn.sql("CREATE TABLE SI_VI_AD0 AS SELECT * FROM admatch0")
# conn.sql("CREATE TABLE SI_VI_AD1 AS SELECT * FROM admatch1")


for db in analyzed_rptr_dfs:
    print(db.loc[0,'Library'])
    
#     conn.sql("CREATE TABLE SI_VI_RPTR0 AS SELECT * FROM rptrmatch0")

RPTR_maxiprep_20X20_S11_L004_R1
RPTR_4_30_30X20_S207_L003
RPTR_4_30_16X16_0925_S14
RPTR_3_30_16X16_S15_L004
RPTR_4_15_20X20_S10_L004
RPTR_3_30_30X20_S207_L004
RPTR_4_30_16X16_sequences_R1


In [30]:
analyzed_rptr_dfs[0]

,index,BCs,TBB,Library,Putative_Tile,Putative_AD
0,TGTGGGCATGTTAG,11702164,GGTAATTATTTGTTCTCTGCTGCTGCTCATGTTCATAGATCTTCTA...,RPTR_maxiprep_20X20_S11_L004_R1,GGTAATTATTTGTTCTCTGCTGCTGCTCATGTTCATAGATCTTCTA...,GTATAATTCCA
1,TGGAGAAAGAAAAT,6525635,0,RPTR_maxiprep_20X20_S11_L004_R1,0,0
2,ACTTCTAAATAAGC,3083395,CAATATAGAAGACAACAACCAGTTTCTCCAGCTGTTCAGAATCAAA...,RPTR_maxiprep_20X20_S11_L004_R1,CAATATAGAAGACAACAACCAGTTTCTCCAGCTGTTCAGAATCAAA...,TTTAGATTACA
3,CTCAATCGACAATT,2192286,TTTCCAACTGCTAATATTCCATCTGAGTTCTTTAATTCATTTACTA...,RPTR_maxiprep_20X20_S11_L004_R1,TTTCCAACTGCTAATATTCCATCTGAGTTCTTTAATTCATTTACTA...,GCCGACCCCTT
4,GTGATCGGTAACAC,980766,0,RPTR_maxiprep_20X20_S11_L004_R1,0,0
...,...,...,...,...,...,...
27343,TCGCTTCCTCCCAC,1,0,RPTR_maxiprep_20X20_S11_L004_R1,0,0
27344,TACCAGTGATAGTG,1,ACTGAGAATTGGTTTCCATTGTTTCCAGAATCTACTATTACTAATC...,RPTR_maxiprep_20X20_S11_L004_R1,ACTGAGAATTGGTTTCCATTGTTTCCAGAATCTACTATTACTAATC...,GAAGGCAGTTT
27345,GAAATAACAGGAAG,1,CATAGAAGGAAATCTTCTGTTAATGGTTCTCCAGTTACTAATGCTT...,RPTR_maxiprep_20X20_S11_L004_R1,CATAGAAGGAAATCTTCTGTTAATGGTTCTCCAGTTACTAATGCTT...,CCGCCATCACC
27346,TGTGGCAATGATAG,1,0,RPTR_maxiprep_20X20_S11_L004_R1,0,0
